# Načtení dat

In [1]:
import pandas as pd
import numpy as np
try:
    # Pokud je konfigurační soubor, použij jej. Pro kompilaci pomocí snakemake nebo v příkazové řádce.
    import config
    source_path = config.merge['input_dir']
    output_folder = config.merge['output_dir']
except:
    # Pokud není konfigurační soubor, použij následující nastavení. Pro kompilaci v Jupyteru.
    source_path = "https://euler.mendelu.cz/dynatree/static/public"
    output_folder = "."
    

In [2]:
df_description = pd.read_excel(f"{source_path}/Popis_Babice_VSE.xlsx",
                              sheet_name="Prehledova tabulka_zakludaje", index_col=1, nrows=14
                              ).drop(0)
df_description;

In [3]:
df_dynamics = pd.read_csv(f"{source_path}/FFT_csv_tukey.csv").rename(columns={'peak':'first_frequency'})
df_dynamics = df_dynamics[df_dynamics.probe.isin(["blueMaj","yellowMaj","Elasto(90)"])]
df_dynamics = df_dynamics[df_dynamics.tree != "JD18"]
df_dynamics;

In [4]:
df_static = pd.read_csv(f"{source_path}/anotated_regressions_static.csv", index_col=0)
df_static = df_static[~df_static.Dependent.isin(["Force(100)","M_Pt"])]
df_static = df_static[df_static.optics==False]
df_static.Slope = np.abs(df_static.Slope)
df_static = df_static[df_static.failed == False]
df_static;

In [5]:
# TODO: pridat Patrikovy data
df_damping = pd.read_csv(f"{source_path}/damping_factor.csv")
df_damping = df_damping[["type", "day", "tree", "probe", "measurement"]+
               [i for i in df_damping.columns if "_LDD" in i]]
df_damping = df_damping[df_damping.tree != "JD18"]
df_damping;

In [6]:
df_vlhkosti = pd.read_csv(f"{source_path}/vlhkosti_babice.csv")
df_sondy = pd.read_csv(f"{source_path}/sondy_a_stromy.csv")
df_sondy;

In [7]:
df_penetrologger = pd.read_csv(f"{source_path}/penetrologger.csv")
df_penetrologger;

In [8]:
df_velocity = pd.read_csv(f"{source_path}/velocity_babice_oprava.csv")
df_velocity = df_velocity[df_velocity["tree"] != "JD18"]

# ze dne odebrat hodiny
df_velocity["day"] = df_velocity["day"].apply(lambda x:x.split( )[0])
df_velocity = df_velocity.drop(["day real", "day"], axis=1) # odebrat nepotrebne sloupce
df_velocity = df_velocity.groupby("tree").mean().reset_index() # prumer za kazdy strom
df_velocity;

# Dendrologické parametry

In [9]:
rename_columns = {
    'tree no:':'tree', 
    'mass Krejza fresh (kg):':"fresh_mass",
    "kategorie":"size_category",
    "diameter at 1.3 m (cm):":"DBH",
    "height (m):": "height",
    "centre of gravity (m)*:": "CG",
    'trunk height (m):': "trunk_height"
}
df_final_description = df_description.rename(columns=rename_columns)[rename_columns.values()]
df_final_description["tree"] = df_final_description["tree"].map(lambda x: f"BK{x:02}")

df_final_description;

# Zpracování penetrologgeru a vlhkosti půdy

Pro kazdy strom nebo a pro kazdy den najdeme jednu hodnotu reprezentujici mechanicke vlastnosti pudy. 
Najdeme median pres vsechny udaje pro danou hloubku, den a strom a pote pro dany den a strom median pro vsechnhy hloubky. 
Prvni krok zajisti, ze vsechny hloubky se budou brat stejnou vahou. Jinak by byla povrchova informace vice zastoupena. 

Pracujeme s hloubkou jenom do 15 cm. Tato hloubka byla urcena po vyhodnoceni dat.

Pocita se podmineny median, pokud jsou k dispozici aspon tri hodnoty.

In [10]:
last_column = 15

df = df_penetrologger.copy()
days = df["day"].drop_duplicates()
trees = df["tree"].drop_duplicates()
trees = [i for i in trees if (i!="JD18") & (i!="BK25")]
df = df[df["tree"].isin(trees)]

df = df[~((df["tree"] == "BK10") & (df["day"] == "2021-03-22"))]
df = df.drop(["0","směr","poznámka","PENETRATION DATA"], axis=1)
days = df["day"].drop_duplicates().to_numpy()
df = df.loc[:,:f"{last_column}"]


# Funkce pro výpočet mediánu při alespoň 3 nenan hodnotách
def conditional_median(row):
    valid_values = row.dropna()  # Odstranění NaN hodnot
    if len(valid_values) >= 3:
        return valid_values.median()
    return np.nan  # Pokud méně než 3 hodnoty, vrátí NaN


df_means = df.groupby(["day","tree"]).agg(conditional_median)
df_final_penetro = df_means.median(axis=1, skipna=True)
df_final_penetro.name="pressure"
df_final_penetro = df_final_penetro.reset_index()
df_final_penetro;

# Přidání vlhkostí k penetrologgeru

Přidáváme první dva horizonty`

In [11]:
df_m = df_vlhkosti.copy()
df_m = df_m [["vzorek","hmotnostní vlhkost  w", "den"]]
df_m.columns = ["vzorek","w", "den"]
# ve soupci vzorek rozdelit zapisy typu 1A na dva sloupce, sonda a horizont.
df_m["horizont"] = df_m["vzorek"].apply(lambda x: x[-1])
df_m["sonda"] = df_m["vzorek"].apply(lambda x: x[:-1]).astype(int)

# načtení tabulky pro převod mezi stromy a sondami
sondy = df_sondy
sondy;

In [12]:
sondy["strom"] = sondy["strom"].apply(lambda x: "JD18" if x==18 else f"BK{x:02}")
df_m = df_m.merge(sondy, on='sonda', how='left')

# df_m = df_m[df_m["strom"]!="JD18"]
# df_m[df_m["den"]=="2021-03-22"]

df_m = (df_m[df_m["horizont"]
        .isin(["A","B"])]  # vybere prvni dva horizonty
        .drop(["vzorek", "horizont","sonda"], axis=1) # vynechat sloupce
        .groupby(["den","strom"]).mean()   # prumer 
        .reset_index()
       )
df_m.columns = ["day","tree","w"]
# df_m[df_m["day"]=="2021-03-22"]

In [13]:
df_final_penetro_moisture = df_final_penetro.merge(df_m, on=["day","tree"], how='outer')
df_final_penetro_moisture = df_final_penetro_moisture[df_final_penetro_moisture.tree != "JD18"]
df_final_penetro_moisture;

Data rozdelit na tri skupiny, protoze se opakuje datum:

a) 2024-09-02, b) 2024-29-20_mokro a c) zbytek

skupina c) se bude slucovat podle datumu, skupiny a) a b) podle datumu a typu mereni.


In [14]:
a = df_final_penetro_moisture[df_final_penetro_moisture.day == "2024-09-02"]
b = df_final_penetro_moisture[df_final_penetro_moisture.day == "2024-09-02_mokro"]
c = df_final_penetro_moisture[~df_final_penetro_moisture.day.isin(["2024-09-02","2024-09-02_mokro"])]
b.loc[:,["day","type"]] = ["2024-09-02", "mokro"]
df_final_penetro_moisture_a = a.merge(pd.DataFrame({'day':["2024-09-02"]*2, 'type':["normal","afterro2"]}), how='left')
df_final_penetro_moisture_b = b
df_final_penetro_moisture_c = c

# Tahovky převést na camera/nocamera

In [15]:
# Limity jsou stanoveny podle analýzy dat pomocí DBSCAN a IQR
R2_limit_M = 0.95
R2_limit_M_Elasto = 0.97

df_tahovky = df_static.copy()  # zkopirovat si, aby zustala i puvodni data
df_tahovky = df_tahovky[df_tahovky["Dependent"] != "M_Pt"]  # vynechat optiku
df_tahovky = df_tahovky[df_tahovky["Dependent"] != "Force(100)"]  # vynechat siloměr
df_tahovky = df_tahovky[df_tahovky["tree"] != "JD18"]  # vynechat jedličku
df_tahovky = df_tahovky[~df_tahovky["Slope"].isna()]  # nesmí být nan v Slope
df_tahovky = df_tahovky[df_tahovky["optics"] == False] # vynechat optiku, pro jistotu ještě jednou

# odfiltrovat spatne R^2 podle vysledku DBSCAN a hlavne IQR
mask = (df_tahovky["Dependent"] == "M") & (df_tahovky["R^2"] < R2_limit_M)
df_tahovky = df_tahovky[~mask]
mask = (df_tahovky["Dependent"] == "M_Elasto") & (df_tahovky["R^2"] < R2_limit_M_Elasto)
df_tahovky = df_tahovky[~mask]

df_tahovky["probe"] = df_tahovky["Independent"].apply(lambda x: "Elasto(90)" if x=="Elasto-strain" else x)

# Nastaveni Camera/NoCamera. Nechat i puvodni kvuli slucovani dat
mask = df_tahovky["probe"].isin(["blueMaj", "yellowMaj"])   # radky s inklinometry
df_tahovky.loc[:,"probeCam"] = df_tahovky["probe"]
df_tahovky.loc[mask,"probeCam"] = df_tahovky.loc[mask,"kamera"].apply(lambda x: "InclinoCamera" if x is True else "InclinoNoCamera" if x is False else x)  # prejmenovani

df_tahovky = df_tahovky[["type","day","tree","probe", "probeCam","measurement","pullNo","Slope"]]

# Slope kazdeho inklinometru do samostatneho sloupce
probes = ["InclinoNoCamera", "InclinoCamera", "Elasto(90)"]
columns = ["slope_root_stiff_nocam", "slope_root_stiff_cam", "slope_stem_stiff"]
for probe, column in zip(probes, columns):
    mask = df_tahovky["probeCam"]==probe
    df_tahovky.loc[mask, column] = df_tahovky.loc[mask,"Slope"]
df_tahovky;

# Finální spojení

TODO

* Co znamená v mých poznámkách (Robert) Sapflow se tremi vykricniky?
* Dodat data od Patrika pro damping pomocí FFT
* leaves True/False

In [16]:
# df_final_penetro_moisture_a
# df_final_penetro_moisture_b
# df_final_penetro_moisture_c
# df_final_description
# df_velocity
# df_damping
# df_tahovky
# df_dynamics

In [17]:
(df_tahovky.shape, 
 df_dynamics.shape, 
 [i for i in df_tahovky.columns if i in df_dynamics.columns],
 [i for i in df_dynamics.columns if i in df_tahovky.columns]
)

((2548, 11),
 (1374, 6),
 ['type', 'day', 'tree', 'probe', 'measurement'],
 ['type', 'day', 'tree', 'measurement', 'probe'])

In [18]:
# spojit tahovky a dynamiku pres spolecne sloupce
# 'type', 'day', 'tree', 'measurement', 'probe'
# Rozmery uplne nesedi, protoze tahovky nekdy byly vyhozeny a dynamika ne, nebo naopak.
DF = df_tahovky.merge(df_dynamics, how='outer')
DF.shape

(2727, 12)

In [19]:
[i for i in DF if i in df_final_description]

['tree']

In [20]:
# Pridat popis stromu, rozmery, dendroparametry
DF = DF.merge(df_final_description, how='left')
DF;

In [21]:
# Pridat rychlosti a data od Janka. Data pro strom. 
DF = DF.merge(df_velocity, how='left')

In [22]:
DF;

In [23]:
[i for i in DF.columns if i in df_damping.columns], [i for i in df_damping.columns if i in DF.columns];

In [24]:
# pridat tlumeni podle poli ['type', 'day', 'tree', 'probe', 'measurement']
DF = DF.merge(df_damping, how='left')

In [25]:
# Skupina c) se bude slucovat podle datumu, skupiny a) a b) podle datumu a typu mereni.
# Rozdělit na tri disjunktní skupiny, každou spojit se správnými daty a potom skupiny sesypat dohromady
# Je to kvůli tomu, že se nedá použít merge když sloupec už existuje a update mi moc nefungovalo (Robert)
mask_a = (DF.day == "2024-09-02") & (DF.type != "mokro")
mask_b = (DF.day == "2024-09-02") & (DF.type == "mokro")
mask_c = (DF.day != "2024-09-02")
DF_a = DF[mask_a]
DF_b = DF[mask_b]
DF_c = DF[mask_c]
DF_full = pd.concat(
    [DF_a.merge(df_final_penetro_moisture_a, how='left'),
     DF_b.merge(df_final_penetro_moisture_b, how='left'),
     DF_c.merge(df_final_penetro_moisture_c, how='left')
    ])
DF_full
    

,type,day,tree,probe,probeCam,measurement,pullNo,Slope,slope_root_stiff_nocam,slope_root_stiff_cam,...,w_all,dens_e,w_e,maxima_LDD,hilbert_LDD,wavelet_LDD,def_LDD,defmulti_LDD,pressure,w
0,afterro2,2024-09-02,BK01,Elasto(90),Elasto(90),M01,0.0,195667.632503,NaN,NaN,...,66.263333,1046.383333,65.773333,NaN,NaN,NaN,NaN,NaN,4.2,8.262565
1,afterro2,2024-09-02,BK01,Elasto(90),Elasto(90),M01,1.0,197627.290150,NaN,NaN,...,66.263333,1046.383333,65.773333,NaN,NaN,NaN,NaN,NaN,4.2,8.262565
2,afterro2,2024-09-02,BK01,Elasto(90),Elasto(90),M01,2.0,197600.175867,NaN,NaN,...,66.263333,1046.383333,65.773333,NaN,NaN,NaN,NaN,NaN,4.2,8.262565
3,afterro2,2024-09-02,BK01,Elasto(90),Elasto(90),M02,0.0,185751.348242,NaN,NaN,...,66.263333,1046.383333,65.773333,0.466268,0.457841,0.372547,0.436636,0.451026,4.2,8.262565
4,afterro2,2024-09-02,BK01,Elasto(90),Elasto(90),M03,0.0,199603.985111,NaN,NaN,...,66.263333,1046.383333,65.773333,0.434982,0.434297,0.356758,0.431574,0.428556,4.2,8.262565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2184,normal,2024-04-10,BK24,yellowMaj,InclinoCamera,M01,2.0,754.643477,NaN,754.643477,...,50.200000,1089.006667,54.566667,NaN,NaN,NaN,NaN,NaN,1.3,38.149670
2185,normal,2024-04-10,BK24,yellowMaj,InclinoCamera,M02,0.0,741.903050,NaN,741.903050,...,50.200000,1089.006667,54.566667,NaN,NaN,NaN,NaN,NaN,1.3,38.149670
2186,normal,2024-04-10,BK24,yellowMaj,InclinoCamera,M03,0.0,690.372862,NaN,690.372862,...,50.200000,1089.006667,54.566667,NaN,NaN,NaN,NaN,NaN,1.3,38.149670
2187,normal,2024-04-10,BK24,yellowMaj,InclinoCamera,M04,0.0,714.670399,NaN,714.670399,...,50.200000,1089.006667,54.566667,NaN,NaN,NaN,NaN,NaN,1.3,38.149670


In [26]:
# DF_full.loc[DF.day == "2024-09-02",["day","type","pressure","w"]]


In [27]:
lambda_i = 1.8751
DF_full["m"] = DF_full["fresh_mass"]
DF_full["EdynL"] = DF_full["CL"]**2 * DF_full["dens_e"]
DF_full["Model1"] = np.sqrt(1/DF_full["height"])
DF_full["Model1_CG"] = np.sqrt(1/DF_full["CG"])

DF_full["Model2"] = DF_full.DBH/DF_full.height**2
DF_full["Model2_CG"] = DF_full.DBH/DF_full.CG**2

DF_full["I"] = np.pi * DF_full["DBH"]**4 / 64
DF_full["Model3"] = np.sqrt(DF_full["EdynL"]*DF_full["I"]/DF_full["fresh_mass"]) 
DF_full["Model4"] = DF_full["Model3"] * lambda_i**2 / (2*np.pi*(DF_full["CG"])**2)
DF_full["Model5"] = np.sqrt(DF_full["Model2"])
DF_full["Model6"] = np.sqrt(DF_full.DBH/DF_full.CG**2/DF_full["fresh_mass"])

DF_full["slenderness_CG"] = DF_full["DBH"]/DF_full["CG"]
DF_full["slenderness_H"] = DF_full["DBH"]/DF_full["height"]
DF_full["m_CG^2"] = DF_full["m"] * DF_full["CG"]**2



In [28]:
DF_full.to_csv(f"{output_folder}/dynatree_data_merge.csv")

In [30]:
DF_full.columns

Index(['type', 'day', 'tree', 'probe', 'probeCam', 'measurement', 'pullNo',
       'Slope', 'slope_root_stiff_nocam', 'slope_root_stiff_cam',
       'slope_stem_stiff', 'first_frequency', 'fresh_mass', 'size_category',
       'DBH', 'height', 'CG', 'trunk_height', 'CL', 'CR', 'CT', 'dens_all',
       'w_all', 'dens_e', 'w_e', 'maxima_LDD', 'hilbert_LDD', 'wavelet_LDD',
       'def_LDD', 'defmulti_LDD', 'pressure', 'w', 'm', 'EdynL', 'Model1',
       'Model1_CG', 'Model2', 'Model2_CG', 'I', 'Model3', 'Model4', 'Model5',
       'Model6', 'slenderness_CG', 'slenderness_H', 'm_CG^2'],
      dtype='object')